<a href="https://colab.research.google.com/github/himanshudas13/Land-Use-Land-Cover/blob/master/Land-Use-Land-Cover%20/Prediction/lulc_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, LSTM, TimeDistributed, Dense, Reshape, Input,Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from PIL import Image, ImageDraw
import os
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
from tensorflow.keras import layers, models
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, MaxPooling2D, Dense, Flatten, Reshape, Input
from tensorflow.keras.models import Model

In [ ]:
# load saved images

input_shape = (448, 448, 3)  # Desired shape of input images
time_steps = 2  # Number of time steps in the series
data_dir = '/content/Classified'


def load_and_preprocess_images(data_dir, input_shape):
    images = []
    for filename in sorted(os.listdir(data_dir)):
        if filename.endswith('.tif') or filename.endswith('.png'):
            img_path = os.path.join(data_dir, filename)
            img = cv2.imread(img_path)
            print(img.shape)
            img = cv2.resize(img, (input_shape[0], input_shape[1]))
            plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))  # Convert from BGR to RGB for Matplotlib
            plt.show()
            img = img / 255.0  # Normalize to [0, 1]
            print(f'{img.min()} n {img.max()}')

            images.append(img)
    return np.array(images)

# Load images
images = load_and_preprocess_images(data_dir, input_shape)

In [ ]:
# split train and test set
image_train=images[1:4]
image_test=images[3:5]

In [ ]:
def prepare_data_for_lstm( encoded_features, time_steps):

    # Prepare time series data
    X, y = [], []
    for i in range(len(encoded_features) - time_steps):
        # Input sequence (time_steps images)
        X.append(encoded_features[i:i + time_steps])

        # Output (next feature vector or target)
        y.append(encoded_features[i + time_steps])

    return np.array(X), np.array(y)

In [ ]:
X_train, y_train = prepare_data_for_lstm(image_train, time_steps)
print(X_train.shape)
print(y_train.shape)

In [ ]:
X_test, y_test = prepare_data_for_lstm(image_test, time_steps)
print(X_test.shape)
print(y_test.shape)

In [ ]:
def create_cnn_encoder(input_shape):
    model = models.Sequential()

    model.add(layers.InputLayer(input_shape=input_shape))

    # Apply Conv2D and MaxPooling2D inside TimeDistributed
    model.add(layers.TimeDistributed(layers.Conv2D(8, (3, 3), activation='relu', padding='same')))
    model.add(layers.TimeDistributed(layers.MaxPooling2D((2, 2))))

    # model.add(layers.TimeDistributed(layers.Conv2D(16, (3, 3), activation='relu', padding='same')))
    # model.add(layers.TimeDistributed(layers.MaxPooling2D((2, 2))))

    model.add(layers.TimeDistributed(layers.Conv2D(32, (3, 3), activation='relu', padding='same')))
    model.add(layers.TimeDistributed(layers.MaxPooling2D((2, 2))))

    # Flatten the features
    model.add(layers.TimeDistributed(layers.Flatten()))

    # Reduce dimensions to 16
    model.add(layers.TimeDistributed(layers.Dense(32, activation='relu')))

    return model



In [ ]:
input_shape = ( 2, 448, 448, 3)  # Sequence length of 2, image size 128x128, 3 channels
cnn_encoder = create_cnn_encoder(input_shape)
cnn_encoder.summary()

In [ ]:
img_encoded=cnn_encoder.predict(X_train)
print(img_encoded.shape)

In [ ]:
def create_lstm_model(encoded_feature_shape, time_steps):
    model = models.Sequential()

    # Input Layer
    model.add(layers.Input(shape=(time_steps, *encoded_feature_shape)))

    # RNN layer 1 with reduced units
    model.add(layers.SimpleRNN(16, return_sequences=True, activation='relu'))

    # RNN layer 2 with reduced units
    model.add(layers.SimpleRNN(8, return_sequences=True, activation='relu'))

    # RNN layer 3 with 16 units to output a feature vector
    model.add(layers.SimpleRNN(32, return_sequences=False, activation='relu'))

    return model

In [ ]:
from tensorflow.keras import layers, models

from tensorflow.keras import layers, models

def create_decoder_model(latent_dim, output_shape=(452, 452, 3)):
    model = models.Sequential()

    # Input Layer
    model.add(layers.InputLayer(input_shape=(latent_dim,)))

    # Dense Layer to expand back to a higher-dimensional feature map
    model.add(layers.Dense(56 * 56 * 64, activation='relu'))  # Adjust size to match an intermediate feature map size
    model.add(layers.Reshape((56, 56, 64)))  # Reshape to (56, 56, 64)

    # Upsampling and Conv2D Layers to progressively reconstruct the image
    model.add(layers.UpSampling2D((2, 2)))  # (56, 56, 64) -> (112, 112, 64)
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))

    model.add(layers.UpSampling2D((2, 2)))  # (112, 112, 64) -> (224, 224, 64)
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))

    model.add(layers.UpSampling2D((2, 2)))  # (224, 224, 64) -> (448, 448, 64)
    model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))

    # Adjust to the target size
    model.add(layers.Cropping2D(cropping=((0, 0), (0, 0))))  # Crop to (452, 452)
    model.add(layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same'))  # Output shape (452, 452, 3)

    return model


In [ ]:
decoder_model = create_decoder_model(decoder_input.shape[1], ( 452, 452, 3))
decoder_model.summary()

In [ ]:
predicted_image=decoder_model.predict(decoder_input)
print(predicted_image.shape)

In [ ]:

# Create the full model
def create_full_model(cnn_encoder, lstm_model, decoder_model, time_steps):

    cnn_input = layers.Input(shape=input_shape)

    cnn_output = cnn_encoder(cnn_input)



    # cnn_output_reshaped = layers.Reshape((time_steps, *cnn_encoder.output_shape[1:]))(cnn_output)
    # # task 3: no need to reshape u can directly take cnn_encoder_output


    # Pass through LSTM
    lstm_output = lstm_model(cnn_output)

    # Pass through Decoder
    decoded_image = decoder_model(lstm_output)

    # Define the full model
    full_model = models.Model(inputs=cnn_input, outputs=decoded_image)

    return full_model

full_model = create_full_model(cnn_encoder, lstm_model, decoder_model, time_steps)

from tensorflow.keras.optimizers import Adam

learning_rate = 0.001  # Adjust this value based on your needs

# Create the Adam optimizer with the custom learning rate
optimizer = Adam(learning_rate=learning_rate)

# Compile and train the full model
full_model.compile(optimizer=optimizer, loss='mean_squared_error')
full_model.fit(X_train,y_train,epochs=20, batch_size=10, validation_split=0.0)


